In [1]:
!pip3 install 'torch==0.3.1'
!pip3 install 'tensorboardX'

     |████████████████████████████████| 496.4MB 32kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 0.3.1 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.3.1 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101


In [2]:
# Load the Drive helper and mount
from google.colab import drive
import os
drive.mount('/content/drive')

path = 'drive/My Drive/ego-rnn/'
os.chdir(path)
cwd = os.getcwd()
print("Current dir: "+cwd)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current dir: /content/drive/My Drive/ego-rnn


In [3]:
from __future__ import print_function, division
from flow_resnet import *
from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
from tensorboardX import SummaryWriter
import torch.nn as nn
from torch.autograd import Variable
from makeDatasetFlow import *
import argparse
import sys

In [4]:
data_Dir = "GTEA61/"
#user_train =  ['S1','S3','S4']
#user_val = ['S2']
outDir = 'experiments'
stackSize = 5 #frame
trainBatchSize = 32
valBatchSize = 32
num_classes = 61

numEpochs = 750
lr1 = 1e-2
decay_factor = 0.5
decay_step = [150,300,500]

In [5]:
min_accuracy = 0

model_folder = os.path.join('./', outDir, 'flow-t')  # Dir for saving models and log files
# Create the dir
if os.path.exists(model_folder):
    print('Dir {} exists!'.format(model_folder))
    #sys.exit()
#os.makedirs(model_folder)

# Log files
writer = SummaryWriter(model_folder)
train_log_loss = open((model_folder + '/train_log_loss.txt'), 'w')
train_log_acc = open((model_folder + '/train_log_acc.txt'), 'w')
val_log_loss = open((model_folder + '/val_log_loss.txt'), 'w')
val_log_acc = open((model_folder + '/val_log_acc.txt'), 'w')

Dir ./experiments/flow-t exists!


In [6]:
# Data loader
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

spatial_transform = Compose([Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                              ToTensor(), normalize])

vid_seq_train = makeDataset(cwd + '/' + data_Dir, spatial_transform=spatial_transform, sequence=False,
                            stackSize=stackSize, fmt='.png')

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize,
                            shuffle=True, sampler=None, num_workers=4, pin_memory=True) 
   
vid_seq_val = makeDataset(data_Dir, spatial_transform=Compose([Scale(256), CenterCrop(224), ToTensor(), normalize]),
                            sequence=False, stackSize=stackSize, fmt='.png', phase='Test')

val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=valBatchSize,
                        shuffle=False, num_workers=2, pin_memory=True)

valInstances = vid_seq_val.__len__()
trainInstances = vid_seq_train.__len__()
print('Number of samples in the dataset: training = {} | validation = {}'.format(trainInstances, valInstances))


S1
114 60
S2
S3
227 60
S4
341 60
S1
S2
116 60
S3
S4
Number of samples in the dataset: training = 341 | validation = 116


In [7]:
model = flow_resnet34(True, channels=2*stackSize)
model.train(True)
train_params = list(model.parameters())

model.cuda()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:00<00:00, 119513079.33it/s]


ResNet(
  (conv1): Conv2d(10, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (

In [8]:
loss_fn = nn.CrossEntropyLoss()

optimizer_fn = torch.optim.SGD(train_params, lr=lr1, momentum=0.9, weight_decay=5e-4)

optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step, gamma=decay_factor)

In [9]:
train_iter = 0

for epoch in range(numEpochs):
    optim_scheduler.step()
    epoch_loss = 0
    numCorrTrain = 0
    trainSamples = 0
    iterPerEpoch = 0
    model.train(True)
    writer.add_scalar('lr', optimizer_fn.param_groups[0]['lr'], epoch+1)
    for i, (inputs, targets) in enumerate(train_loader):
        train_iter += 1
        iterPerEpoch += 1
        optimizer_fn.zero_grad()
        inputVariable = Variable(inputs.cuda())
        labelVariable = Variable(targets.cuda())
        trainSamples += inputs.size(0)

        output_label, _ = model(inputVariable)
        loss = loss_fn(output_label, labelVariable)
        loss.backward()
        optimizer_fn.step()
        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += (predicted == targets.cuda()).sum()
        epoch_loss += loss.data[0]
    avg_loss = epoch_loss/iterPerEpoch
    trainAccuracy = (numCorrTrain / trainSamples) * 100
    print('Train: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch + 1, avg_loss, trainAccuracy))
    writer.add_scalar('train/epoch_loss', avg_loss, epoch+1)
    writer.add_scalar('train/accuracy', trainAccuracy, epoch+1)
    train_log_loss.write('Training loss after {} epoch = {}\n'.format(epoch+1, avg_loss))
    train_log_acc.write('Training accuracy after {} epoch = {}\n'.format(epoch+1, trainAccuracy))
    if (epoch+1) % 1 == 0:
        model.train(False)
        val_loss_epoch = 0
        val_iter = 0
        val_samples = 0
        numCorr = 0
        for j, (inputs, targets) in enumerate(val_loader):
            val_iter += 1
            val_samples += inputs.size(0)
            inputVariable = Variable(inputs.cuda(), volatile=True)
            labelVariable = Variable(targets.cuda(async=True), volatile=True)
            output_label, _ = model(inputVariable)
            val_loss = loss_fn(output_label, labelVariable)
            val_loss_epoch += val_loss.data[0]
            _, predicted = torch.max(output_label.data, 1)
            numCorr += (predicted == targets.cuda()).sum()
        val_accuracy = (numCorr / val_samples) * 100
        avg_val_loss = val_loss_epoch / val_iter
        print('Validation: Epoch = {} | Loss = {} | Accuracy = {}'.format(epoch + 1, avg_val_loss, val_accuracy))
        writer.add_scalar('val/epoch_loss', avg_val_loss, epoch + 1)
        writer.add_scalar('val/accuracy', val_accuracy, epoch + 1)
        val_log_loss.write('Val Loss after {} epochs = {}\n'.format(epoch + 1, avg_val_loss))
        val_log_acc.write('Val Accuracy after {} epochs = {}%\n'.format(epoch + 1, val_accuracy))
        if val_accuracy > min_accuracy:
            save_path_model = (model_folder + '/model_flow_state_dict.pth')
            torch.save(model.state_dict(), save_path_model)
            min_accuracy = val_accuracy


train_log_loss.close()
train_log_acc.close()
val_log_acc.close()
val_log_loss.close()
writer.export_scalars_to_json(model_folder + "/all_scalars.json")
writer.close()

Train: Epoch = 1 | Loss = 4.428259979594838 | Accuracy = 3.225806451612903
Validation: Epoch = 1 | Loss = 4.283688485622406 | Accuracy = 4.310344827586207
Train: Epoch = 2 | Loss = 4.4088873863220215 | Accuracy = 4.105571847507331
Validation: Epoch = 2 | Loss = 4.815854966640472 | Accuracy = 5.172413793103448
Train: Epoch = 3 | Loss = 4.266820062290538 | Accuracy = 6.744868035190615
Validation: Epoch = 3 | Loss = 8.580298900604248 | Accuracy = 1.7241379310344827
Train: Epoch = 4 | Loss = 4.198711178519509 | Accuracy = 3.519061583577713
Validation: Epoch = 4 | Loss = 4.442978024482727 | Accuracy = 5.172413793103448
Train: Epoch = 5 | Loss = 4.18736947666515 | Accuracy = 6.451612903225806
Validation: Epoch = 5 | Loss = 4.321431338787079 | Accuracy = 6.0344827586206895
Train: Epoch = 6 | Loss = 4.102513399991122 | Accuracy = 3.519061583577713
Validation: Epoch = 6 | Loss = 3.9538053274154663 | Accuracy = 4.310344827586207
Train: Epoch = 7 | Loss = 4.063540805469859 | Accuracy = 4.39882697

In [10]:
print('Best accuracy after {} epochs = {}'.format(epoch, min_accuracy))

Best accuracy after 749 epochs = 44.827586206896555
